In [1]:
# run cell once. makes imports refresh every time changes are pulled
%load_ext autoreload
%autoreload 2

In [2]:
# initial clone, run this cell once when starting development/testing
!git clone https://github.com/klabarbara/bank_intent.git
%cd bank_intent

# silently installs dependencies unless error
!pip install -e . > /dev/null 2>&1

Cloning into 'bank_intent'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 133 (delta 67), reused 102 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 32.26 KiB | 16.13 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/bank_intent


In [8]:
# run this cell after local changes are pushed to remote repo
!git pull origin main

# silently installs dependencies unless error
!pip install -e . > /dev/null 2>&1
print("project dependencies reinstalled!")

remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 20 (delta 13), reused 20 (delta 13), pack-reused 0 (from 0)
Unpacking objects: 100% (20/20), 2.83 KiB | 723.00 KiB/s, done.
From https://github.com/klabarbara/bank_intent
 * branch            main       -> FETCH_HEAD
   e1dfce7..6813436  main       -> origin/main
Updating e1dfce7..6813436
Fast-forward
 configs/train.yaml |  2 +-
 src/baseline.py    | 44 +++++++++++++++++++++++++++++---------------
 src/eval.py        |  4 +---
 src/gradio_ui.py   | 42 +++++++++++++++++++++++++++++++++---------
 src/multiregime.py |  2 ++
 5 files changed, 66 insertions(+), 28 deletions(-)
project dependencies reinstalled!


In [3]:
from src.etl import run as etl_run
etl_run()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

ETL complete.


In [4]:
# exposes MLflow UI over HTTP
!pip install pyngrok
from pyngrok import ngrok
import os

# requires an ngrok account and auth token: https://dashboard.ngrok.com/
os.environ["NGROK_AUTH_TOKEN"] = input("Enter your ngrok auth token: ")

# use in command
!ngrok config add-authtoken $NGROK_AUTH_TOKEN

ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel.public_url)  # url should end in ngrok-free.app

Enter your ngrok auth token: 311LcZNaFke1iaRhx1C3HyZXIRc_cC9mXacKQ8co77Wfe81P
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
https://050185d7f9de.ngrok-free.app


In [5]:
import subprocess

# launch MLflow UI asynchronously
subprocess.Popen([
    "mlflow", "ui",
    "--backend-store-uri", "mlruns",
    "--port", "5000"
])

print("MLflow UI started in background on port 5000")

MLflow UI started in background on port 5000


In [ ]:
# various training runs

# smoke test running on 10 rows
# !python -m src.baseline --sample-n 10
!python -m src.baseline
# !python -m src.sft_lora
# !python -m src.multiregime -r head-only
# !python -m src.multiregime -r lora
# !python -m src.multiregime -r full-ft

2025-08-18 15:42:57.274055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755531777.294383   21104 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755531777.300906   21104 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755531777.318039   21104 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755531777.318069   21104 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755531777.318073   21104 computation_placer.cc:177] computation placer alr

In [ ]:
# !zip -r mlruns.zip /content/bank_intent/bank_intent/mlruns
# from google.colab import files
# files.download("mlruns.zip")

In [ ]:
import pandas as pd
df = pd.read_parquet("data/processed/train.parquet")
label_names = sorted(df["label_text"].unique())
print(label_names)

In [7]:
# default path
!python -m src.eval  --checkpoint-path artifacts/models/deberta_lora --data-path data/processed/test.parquet

# CPU only
# !python -m src.eval  --device-id -1

2025-08-18 14:37:41.074069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755527861.094796    4309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755527861.101303    4309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755527861.118398    4309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755527861.118431    4309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755527861.118437    4309 computation_placer.cc:177] computation placer alr

In [ ]:
from transformers import AutoConfig, AutoTokenizer

checkpoint = str("/content/bank_intent/artifacts/models/deberta_lora/")

config = AutoConfig.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

print("Tokenizer limit:", tokenizer.model_max_length)
print("Model positional limit:", getattr(config, "max_position_embeddings", None))


In [ ]:
!uvicorn backend:app --reload

In [ ]:
!python src/gradio_ui.py

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c31eb8a80d883c67d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3125, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/bank_intent/src/gradio_ui.py", line 12, in <module>
    iface.launch(share=True)
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3031, in launch
    self.block_thread()
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3129, in block_thread
    self.server.close()
  File "/usr/local/lib/p